# 1 导入库

In [3]:
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import tokenizer_from_json
from nltk import word_tokenize 
import nltk
nltk.download('punkt')
import json
import numpy as np

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


# 2 数据加载

In [28]:
from google.colab import drive
drive.mount('/content/gdrive')


Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [29]:
import os
#os.chdir("/content/drive/Shared drives/项目名")
os.chdir("/content/gdrive/MyDrive/ai_lab_final")

## 2.1 加载分词字典

In [6]:
json_file = open('./model/token_config_lstm.json', 'r', encoding = "ISO-8859-1")
token_config = json.load(json_file)
tokenizer = tokenizer_from_json(token_config)

In [7]:
model = load_model('./model/lstm_model.h5')

# 3 定义预测函数

In [8]:
# 情感预测
def predict(text):
    
    # 对句子分词
    cw = list(word_tokenize(text)) 
    
    # list转字符串，元素之间用' '隔开
    texts = ' '.join(cw)
    
    # 把词转换为编号，编号大于30000的词会被过滤掉
    sequences = tokenizer.texts_to_sequences([texts]) 
    
    # model.input_shape为(None, 202)，202为训练模型时的序列长度
    # 把序列设定为202的长度，超过202的部分舍弃，不到202则补0
    sequences = pad_sequences(sequences, maxlen=model.input_shape[1], padding='pre')
    
    # 模型预测
    result = np.argmax(model.predict(sequences))
    return result

# 4 预测推理

使用 `predict` 函数对新数据进行预测：

In [9]:
import csv
import pandas as pd
 
data = pd.read_csv('test.csv',encoding = "ISO-8859-1")
#print(list(data['sentence'])) #test


In [10]:
resultVector=[]

for text in list(data['sentence']):
    resultVector.append(predict(text)) 

In [11]:
print(len(resultVector))

7000


In [30]:
#with open('test.csv','r',encoding='gbk') as csvFile:  #此处的csv是源表，即想要写入的表
with open('test.csv','r',encoding='ISO-8859-1') as csvFile:  #此处的csv是源表，即想要写入的表
    rows = csv.reader(csvFile)
    with open('20337188_denglihua.csv','w',newline='') as f: #这里的csv则是最后输出得到的新表
      writer = csv.writer(f)
      i = 0
      for row in rows:
        if i==0:
          i = i + 1
          row[len(row)-1]='gold_label'
          writer.writerow(row)
          continue
        row[len(row)-1]=resultVector[i-1]
        #row[len(row)-1]=''
        #print(i)
        i = i + 1
        writer.writerow(row)
